# 5. K-Means Clustering with the Retail Data-Set #
Demonstration of using MMlib to apply the K-Means algorithm to Retail data-set.

In [50]:
# Connect to Spark Server
from pyspark.sql import SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook-5").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", "file:///opt/workspace/events").\
        getOrCreate() 

#### Load Data ####

Use `spark.read` to load some data. https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html  
Note on the SQL `col` function: https://stackoverflow.com/questions/40163106/cannot-find-col-function-in-pyspark

In [51]:
# import date_format fuction and col ("return a column") function
from pyspark.sql.functions import date_format, col

In [52]:
# Load data
retailDataFrame = spark.read.option("inferSchema", True).option("header", True).csv("/opt/workspace/datain/retail-data/by-day/*.csv")

Summary of DataFrame methods: https://spark.apache.org/docs/1.6.1/api/java/org/apache/spark/sql/DataFrame.html

In [53]:
# Fill Na's and create a day_of_week column.   coalesce(2) function coalesce's data down to 2 node partitions
preppedDataFrame = retailDataFrame.na.fill(0).withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE")).coalesce(2)

#### Explore the Data ####
Very brief look at the data-set.

In [54]:
preppedDataFrame.take(5)

[Row(InvoiceNo='580538', StockCode='23084', Description='RABBIT NIGHT LIGHT', Quantity=48, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.79, CustomerID=14075.0, Country='United Kingdom', day_of_week='Monday'),
 Row(InvoiceNo='580538', StockCode='23077', Description='DOUGHNUT LIP GLOSS ', Quantity=20, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.25, CustomerID=14075.0, Country='United Kingdom', day_of_week='Monday'),
 Row(InvoiceNo='580538', StockCode='22906', Description='12 MESSAGE CARDS WITH ENVELOPES', Quantity=24, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.65, CustomerID=14075.0, Country='United Kingdom', day_of_week='Monday'),
 Row(InvoiceNo='580538', StockCode='21914', Description='BLUE HARMONICA IN BOX ', Quantity=24, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.25, CustomerID=14075.0, Country='United Kingdom', day_of_week='Monday'),
 Row(InvoiceNo='580538', StockCode='22467', Description='GUMBALL COAT RACK

In [55]:
#Summary stats
preppedDataFrame.describe().show()

+-------+------------------+------------------+--------------------+------------------+------------------+------------------+-----------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|    Country|day_of_week|
+-------+------------------+------------------+--------------------+------------------+------------------+------------------+-----------+-----------+
|  count|            541909|            541909|              540455|            541909|            541909|            541909|     541909|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324|4.6111136260826315|11476.974671024102|       null|       null|
| stddev|13428.417280792186|16799.737628427614|                 NaN|218.08115785023375| 96.75985306117823|  6777.90832601304|       null|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -80995|         -110

In [56]:
# Distribution of records by country
from pyspark.sql.functions import desc
preppedDataFrame.groupby('Country').count().sort(desc("count")).show(100,False)

+--------------------+------+
|Country             |count |
+--------------------+------+
|United Kingdom      |495478|
|Germany             |9495  |
|France              |8557  |
|EIRE                |8196  |
|Spain               |2533  |
|Netherlands         |2371  |
|Belgium             |2069  |
|Switzerland         |2002  |
|Portugal            |1519  |
|Australia           |1259  |
|Norway              |1086  |
|Italy               |803   |
|Channel Islands     |758   |
|Finland             |695   |
|Cyprus              |622   |
|Sweden              |462   |
|Unspecified         |446   |
|Austria             |401   |
|Denmark             |389   |
|Japan               |358   |
|Poland              |341   |
|Israel              |297   |
|USA                 |291   |
|Hong Kong           |288   |
|Singapore           |229   |
|Iceland             |182   |
|Canada              |151   |
|Greece              |146   |
|Malta               |127   |
|United Arab Emirates|68    |
|European 

-> The data is heavily skewed to the UK.

In [57]:
# Distinct number of product codes 
preppedDataFrame.select('StockCode').distinct().count()

4070

-> There are 4070 distinct products

#### Split Data into Test and Training Sets ####
*However*, splitting data into test and training sets is not always necessarily required for un-supervised cluster analysis.  However, we want to analysis the cluster separation ("silhouette") and error we get with a new data set based on a training set to determine best number of clusters to choose.

In [58]:
trainDataFrame = preppedDataFrame.where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame.where("InvoiceDate >= '2011-07-01'")
print(trainDataFrame.count())
print(testDataFrame.count())

245903
296006


#### Use `StringIndexer` and `OneHotEncoder` to Encode Day-of-Week and create a Feature `VectorAssembler` ####
Ref: https://spark.apache.org/docs/latest/ml-features

In [59]:
from pyspark.ml.feature import StringIndexer # takes range of string labels and maps to integer values
from pyspark.ml.feature import OneHotEncoder # One-hot-encoding for categorical features; encode as binary encoded vector-col

In [60]:
indexer = StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_index")
encoder = OneHotEncoder().setInputCol("day_of_week_index").setOutputCol("day_of_week_encoded")

Assemble multi-value vector of features including one-hot representation for day-of-week into a vector stored in a single column.

In [61]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler().setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"]).setOutputCol("features")

#### Create a Model Training Pipeline ####
  
Train a K-Means cluster model with *Day-of-Week* and *Unit Price* and *Quantity* in the feature-set

First, fit the transformers to the data-set.  The `StringIndexer` needs to know how many unique values there are to index.  

In [62]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline().setStages([indexer, encoder, vectorAssembler])
fittedPipeline = transformationPipeline.fit(trainDataFrame)
transformedTraining = fittedPipeline.transform(trainDataFrame)
transformedTest = fittedPipeline.transform(testDataFrame)

Note: it woud be possible to also include the model training in the pipeline above.  Instead we next *cache* the pipeline and then experiment with different hyperparameters training a model against the same pipeline.

In [63]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

#### Import KMeans from `pyspark.ml.clustering` and Fit a Model ####
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.clustering.KMeans.html#pyspark.ml.clustering.KMeans.k

In [67]:
# Import the Spark ML Kmeans model
from pyspark.ml.clustering import KMeans

# Evaluate clustering by computing Silhouette score
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

# setK - the number of clusters to create
#kmeans = KMeans().setK(5).setSeed(42)

In [68]:
#kmModel = kmeans.fit(transformedTraining)

In [69]:
#compute the cost with KMeans.computeCost() - cost is within-set sum of squared errors (distances from center)
#print(kmModel.computeCost(transformedTraining))

#### Look for Optimal Number of Clusters ####
Consider **Cluster Separation** and **Standard Error** to evaluate different numbers of clusters to choose.  
  
This can take several minutes to run. *Skip this step and just choose K=8 clusters to avoid the analysis phase* by skipping ahead to the *"Identify Clusters in the Test Data-Set"* section below.

In [ ]:
# Look for optimal K number of clusters from 3 to 20
costs = []
silhouettes = []
for K in range(3,20):
    kmModel = KMeans().setK(K).setSeed(42).fit(transformedTraining)    
    costs.append(kmModel.computeCost(transformedTraining))
    # predict clusters with test data-set and determine level of separatation between clusters (1 is optimal)
    predictions = kmModel.transform(transformedTest)
    silhouette = evaluator.evaluate(predictions)
    #print("Silhouette with squared euclidean distance = " + str(silhouette))
    silhouettes.append(silhouette)    

In [ ]:
# Plot the results
import matplotlib.pyplot as plt
plt.subplot(2,1,1)
plt.plot(range(3,20), costs)
plt.xlabel("number of clusters (\"K\")")
plt.ylabel("Std Error")
plt.title("Distance from Centers")
plt.grid(True)

plt.subplot(2,1,2)
plt.plot(range(3,20), silhouettes)
plt.xlabel("number of clusters (\"K\")")
plt.ylabel("Silhouette")
plt.title("Separation")
plt.grid(True)

plt.subplots_adjust(hspace=1)

plt.show()


Based on the above outcome a value of between 8 and 15 clusters would be appropriate.  For the rest of this notebook, choose **8** clusters for maximum cluster separation and lower std error (overall distance from cluster center).

#### Identify Clusters in the Test data-set ####
Working with the test data-set, generate preditions from the model

In [73]:
kmModel = KMeans().setK(8).setSeed(42).fit(transformedTraining)    
predictions = kmModel.transform(transformedTest)

In [74]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.9985060075601593


#### Analysis ####


In [75]:
predictions.take(5)
#predictions.select("features").take(5)

[Row(InvoiceNo='580538', StockCode='23084', Description='RABBIT NIGHT LIGHT', Quantity=48, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.79, CustomerID=14075.0, Country='United Kingdom', day_of_week='Monday', day_of_week_index=2.0, day_of_week_encoded=SparseVector(5, {2: 1.0}), features=SparseVector(7, {0: 1.79, 1: 48.0, 4: 1.0}), prediction=0),
 Row(InvoiceNo='580538', StockCode='23077', Description='DOUGHNUT LIP GLOSS ', Quantity=20, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.25, CustomerID=14075.0, Country='United Kingdom', day_of_week='Monday', day_of_week_index=2.0, day_of_week_encoded=SparseVector(5, {2: 1.0}), features=SparseVector(7, {0: 1.25, 1: 20.0, 4: 1.0}), prediction=0),
 Row(InvoiceNo='580538', StockCode='22906', Description='12 MESSAGE CARDS WITH ENVELOPES', Quantity=24, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.65, CustomerID=14075.0, Country='United Kingdom', day_of_week='Monday', day_of_week_index=2.0, day_of_

Other class-labels not used in the feature-set
1. StockCode (same as Description?)  
2. CustomerID  
3. Country  


In [76]:

print("Distinct StockCode:", preppedDataFrame.select('StockCode').distinct().count())
print("Distinct CustomerID:", preppedDataFrame.select('CustomerID').distinct().count())
print("Distinct Country:", preppedDataFrame.select('Country').distinct().count())

Distinct StockCode: 4070
Distinct CustomerID: 4373
Distinct Country: 38


#### Question 1: How many distinct clusters do the Countrys belong to? ####


In [86]:
#Pandas version
#pandas_df = predictions.select("StockCode","CustomerID", "Country", "prediction").toPandas()
#pandas_df.groupby('Country')['prediction'].nunique()

#Spark Version
import pyspark.sql.functions as f
#predictions.groupby('Country').agg(f.expr('count(distinct prediction)').alias('n_clusters')).show(100,False)
predictions.groupby('prediction', 'Country').agg(f.expr('count(distinct prediction)').alias('n_clusters')).sort("Country", "prediction").show(100,False)

+----------+--------------------+----------+
|prediction|Country             |n_clusters|
+----------+--------------------+----------+
|0         |Australia           |1         |
|7         |Australia           |1         |
|0         |Austria             |1         |
|0         |Belgium             |1         |
|0         |Canada              |1         |
|0         |Channel Islands     |1         |
|0         |Cyprus              |1         |
|0         |Czech Republic      |1         |
|0         |Denmark             |1         |
|0         |EIRE                |1         |
|7         |EIRE                |1         |
|0         |European Community  |1         |
|0         |Finland             |1         |
|0         |France              |1         |
|4         |France              |1         |
|7         |France              |1         |
|0         |Germany             |1         |
|0         |Greece              |1         |
|0         |Hong Kong           |1         |
|0        

-> All the countries except the UK are only in one or two clusters.  If the data-set wasn't so heavily skewed to UK data, this might be interesting.

In [87]:
# Question 2 - how many items per cluster
predictions.groupby('prediction').count().show()

+----------+------+
|prediction| count|
+----------+------+
|         1|     1|
|         6|     3|
|         5|     1|
|         4|    22|
|         7|    92|
|         2|     1|
|         0|295886|
+----------+------+



-> only a few very special outliers?

In [97]:
# Get the cluster-ids where count is less than 50, look at those items
# To filter on the count, need to rename the count column: https://stackoverflow.com/questions/32119936/dataframe-how-to-groupby-count-then-filter-on-count-in-scala
outliers = predictions.groupBy('prediction').count().withColumnRenamed("count", "n").filter("n <= 50").sort("n").select("prediction")

In [116]:
outliers_list = outliers.select("prediction").rdd.flatMap(lambda x: x).collect()
#for cluster in outliers_list:
#    predictions.where("prediction = $cluster")

AnalysisException: "cannot resolve '`prediction == 2`' given input columns: [day_of_week, InvoiceDate, UnitPrice, Country, Quantity, InvoiceNo, CustomerID, features, day_of_week_index, prediction, Description, day_of_week_encoded, StockCode];;\n'Project ['prediction == 2]\n+- Project [InvoiceNo#9250, StockCode#9251, Description#9252, Quantity#9274, InvoiceDate#9254, UnitPrice#9275, CustomerID#9276, Country#9257, day_of_week#9285, day_of_week_index#9898, day_of_week_encoded#9909, features#9922, UDF(features#9922) AS prediction#14902]\n   +- Project [InvoiceNo#9250, StockCode#9251, Description#9252, Quantity#9274, InvoiceDate#9254, UnitPrice#9275, CustomerID#9276, Country#9257, day_of_week#9285, day_of_week_index#9898, day_of_week_encoded#9909, UDF(named_struct(UnitPrice, UnitPrice#9275, Quantity_double_VectorAssembler_61158599aa80, cast(Quantity#9274 as double), day_of_week_encoded, day_of_week_encoded#9909)) AS features#9922]\n      +- Project [InvoiceNo#9250, StockCode#9251, Descripti

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector, prediction: int]

In [ ]:
retailDataFrame.where(col("StockCode")==23843).show(20)

In [ ]:
spark.stop()